In [3]:
%%time
import numpy as np
import matplotlib.pyplot as plt
from root_pandas import read_root
from sklearn.preprocessing import normalize

plt.rcParams['figure.figsize'] = 12.0, 8.0

Welcome to JupyROOT 6.10/02
CPU times: user 1.5 s, sys: 592 ms, total: 2.09 s
Wall time: 3.17 s


# ABC - Approximate Bayesian Computation, Inference statistical method.???

# Bayesian Deep learning, Conquering Uncertainty

### Write a data generator, read from .root, 1D strings then transform it into 2D images

better memory handling for 256x256 images. storing it on RAM can take up way too much!!

### Not very well verse in writing this type of operations in python. ned to figure out to better handle it.

standard method takes about 2 GB, images x16, 32GB!!!, that too much for any normal operating system's memory can handle.

In [4]:
%%time

from pyts.transformation import GADF,GASF
from sklearn.preprocessing import normalize
from root_numpy import tree2array,array2tree,array2root
import ROOT
import keras

def reader_pmt(path):
    extra = np.arange(4096, 4480)
    x_file = ROOT.TFile(path)
    x_tree = x_file.Get("tree")
    x_pmtall = tree2array(x_tree, branches="PMTALL")
    x = np.delete(x_pmtall, extra, axis=1)
    return x

def label(q,k):
    x = np.zeros(len(q))
    for i in range(len(q)):
        x[i] = k
    return x

def sep(q,k,z):
    y = label(q,k)
    x1, x2 ,x3 = np.split(q,[int(len(q)*0.6),int(len(q)*0.8)])
    y1, y2 ,y3 = np.split(y,[int(len(q)*0.6),int(len(q)*0.8)])
    if z == 0:
        return x1, y1
    if z == 1:
        return x2, y2
    if z == 2:
        return x3, y3
    
def comb(one,two,three,four,five,portion):
    one1,one2 = sep(one,0,portion)
    two1,two2 = sep(two,1,portion)
    three1,three2 = sep(three,2,portion)
    four1,four2 = sep(four,3,portion)
    five1,five2 = sep(five,4,portion)
    six1,six2 = sep(six,5,portion)
    seven1,seven2 = sep(seven,6,portion)
    
    z = np.concatenate((one1,two1,three1,four1,five1,six1,seven1),axis=0)
    y = np.concatenate((one2,two2,three2,four2,five2,six2,seven2),axis=0)
    return z, y

import random
classes = 5
def generator(x):
    while 1:
        alpha = reader_pmt("Alpha_2280_2350.root")
        bad1 = reader_pmt("BAD_DATA_QUALITY_1.root")
        bad2 = reader_pmt("BAD_DATA_QUALITY_2.root")
        piled = reader_pmt("Piled_2.root")
        ls = reader_pmt("LS.root")
        lsbeta = reader_pmt("LS_BETA.root")
        ref = reader_pmt("RefPulse009.root")
        
        z1, z2 = comb(piled,ref,alpha,bad1,bad2,x)
        s = np.random.permutation(np.arange(len(z2)))
        z1 = z1[s]
        z2 = z2[s]
#         print(len(z2))

        gasf = GASF(image_size=64, overlapping=False, scale='-1')
        for i in range(len(z2)):
            p = np.expand_dims(z1[i],axis=0)
            p = gasf.transform(p)
            q = keras.utils.to_categorical(z2[i],classes)
            yield(p,q)
        
# print(len(x_nor))
test = generator(0)
# print(len(test))
print(next(test)[0].shape)
# print(next(test)[1].shape)
# print(next(test))
# print(next(test))
test = generator(1)
# print(len(test))
print(next(test)[0].shape)
# print(next(test)[1].shape)
test = generator(2)
# print(len(test))
print(next(test)[0].shape)
# print(next(test)[1].shape)

(1, 64, 64, 1)
(1, 64, 64, 1)
(1, 64, 64, 1)
CPU times: user 27.8 s, sys: 3.11 s, total: 31 s
Wall time: 32.2 s


In [5]:
import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Embedding, LSTM

from keras.regularizers import l2
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(8, kernel_size=1, activation="relu", input_shape=(64,64,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(16, 1, activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, 1, activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.3))
# model.add(Dense(16,activation="relu"))
# model.add(Dropout(0.3))
# model.add(Dense(8,activation="relu"))
# model.add(Dropout(0.3))
model.add(Dense(classes,activation="softmax"))
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 8)         16        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 16)        144       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        544       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 32)          0         
__________

In [2]:
%%time

train_gen = generator(0)
val_gen = generator(1)
test_gen = generator(2)
steps_t = 17175
steps_vt  = 5727

history = model.fit_generator(train_gen,steps_per_epoch=steps_t, epochs=10, verbose=1, validation_data=val_gen,validation_steps=steps_vt)
model.save("CNN_256_256_5_classes_noLS_LSBETA.h5")
score = model.evaluate_generator(test_gen,steps=8059)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

print(history.history.keys())
plt.figure()
plt.title("Classification accuracy")
plt.xlabel("epoch")
plt.plot(history.history['acc'],label="training accuracy")
plt.plot(history.history['val_acc'],label="Validation accuracy")
plt.legend(loc=4)
plt.show()

plt.figure()
plt.title("Loss function")
plt.xlabel("epoch")
plt.plot(history.history['loss'],label="training loss")
plt.plot(history.history['val_loss'],label="Validation loss")
plt.legend()
plt.show()

print("Classification accuracy =",history.history['val_acc'][-1])

NameError: name 'generator' is not defined

# Old codes, do not run

In [ ]:
%%time
piled = reader_pmt("Piled_2.root")
ls = reader_pmt("LS.root")
lsbeta = reader_pmt("LS_BETA.root")
ref = reader_pmt("RefPulse009.root")

gadf = GADF(image_size=256, overlapping=False, scale='-1')
piled_gadf = gadf.transform(piled)
ls_gadf = gadf.transform(ls)
lsbeta_gadf = gadf.transform(lsbeta)
ref_gadf = gadf.transform(ref)

del piled,ls,lsbeta,ref

CPU times: user 8min 26s, sys: 46.2 s, total: 9min 12s
Wall time: 8min 42s
